In [19]:
import nrrd
import numpy as np
import os
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from skimage.segmentation import mark_boundaries
import cv2
from scipy import ndimage as ndi
from helper import *
import graph_tool.all as gt
import plotly.graph_objects as go
import time
from datetime import datetime


current_directory = os.getcwd()
label_path = f"{current_directory}/data/label/manual_2_label.nrrd"
raw_data_path = f"{current_directory}/data/raw/manual_2_raw.nrrd"

mask_data, mask_header = nrrd.read(label_path)
raw_data, raw_header = nrrd.read(raw_data_path)

In [20]:
#rotate the data for best cut angle and calculate reduced data representations
# axis = 1 #0 for x, 1 for y and 2 for z
# k = 1 # number of times to rotate the array
# mask_data = np.rot90(mask_data, k, axes=(axis, (axis+1)%3))
# raw_data = np.rot90(raw_data, k, axes=(axis, (axis+1)%3))

reduced_mask_data = coarsen_image(mask_data, 3)
reduced_raw_data = coarsen_image(raw_data, 3)
#[0] is full res, each further index is 2x lower res, 256, 128, 64, 32

In [21]:
#create reduced graph
res_index = 2
rotated_directed_graph, r_src, r_tgt, rotated_weights, x_pos, y_pos, z_pos = create_masked_directed_energy_graph_from_mask(reduced_mask_data[res_index])

64 64 64
Time taken to add vertices to coord_to_vertex: 1.0748591423034668
Time taken to add edges to graph: 2.0179619789123535
Time taken to add source and sink nodes: 0.0815286636352539


In [4]:
def calculate_seam_iter(directed_graph, src, tgt, weights, test_size, x_pos, y_pos, z_pos):
    # Compute the residual capactiy of the edges
    stime = time.time()
    res = gt.boykov_kolmogorov_max_flow(directed_graph, src, tgt, weights) #time complexity: edges * vertices^2 * abs(min cut) 
    print("Time taken to calculate max flow boykov:", time.time()-stime)
    # sptime = time.time()
    # res = gt.push_relabel_max_flow(directed_graph, src, tgt, weights) #time complexity: vertices^3 <- scales better for our graph
    # print("Time taken to calculate max flow push relabel:", time.time()-sptime)
    stime = time.time()
    #use the residual graph to get the max flow
    flow = sum(weights[e] - res[e] for e in tgt.in_edges())
    # print("The maximum flow from source to sink is:", flow)
    print("Time taken to calculate max flow:", time.time()-stime, "flow:", flow)
    stime = time.time()
    # Determine the minimum cut partition
    part = gt.min_st_cut(directed_graph, src, weights, res)
    # print("The number of vertices in the partiton:", sum(part))
    print("Time taken to calculate max flow and min cut:", time.time()-stime)
    # Find the boundary vertices
    stime = time.time()
    boundary_vertices = find_boundary_vertices(np.array(directed_graph.get_edges()), part)
    # print("Number of boundary vertices:", len(boundary_vertices))
    print("Time taken to calculate seam:", time.time()-stime)

    shape = (test_size, test_size, test_size)

    # Convert the boundary vertices to a 3D array
    stime = time.time()
    boundary_array = boundary_vertices_to_array_masked(boundary_vertices, shape, 'x', x_pos, y_pos, z_pos)
    # print("Boundary points marked:", np.sum(boundary_array))
    print("Time taken to convert boundary vertices to array:", time.time()-stime)

    return boundary_array, flow

In [5]:
boundary_array, flow = calculate_seam_iter(rotated_directed_graph, r_src, r_tgt, rotated_weights, reduced_mask_data[res_index].shape[0], x_pos, y_pos, z_pos)
b_arr_up = upscale_and_dilate_3d(boundary_array, upscale_factor=4, dilation_amount=1)
print(b_arr_up.shape)

Time taken to calculate max flow boykov: 0.8359119892120361
Time taken to calculate max flow: 0.03977799415588379 flow: 10045
Time taken to calculate max flow and min cut: 0.09181094169616699
edges that cross the partition: 1540352
boundary vertices: 7653
Time taken to calculate seam: 0.01895594596862793
Time taken to convert boundary vertices to array: 0.02890777587890625
(256, 256, 256)


In [6]:
# copying and masking the high res data for masked graph creation
res_index = 0
masked_array = reduced_mask_data[res_index].copy()
print(masked_array.shape, b_arr_up.shape)
masked_array[b_arr_up == 0] = -1

masked_graph, masked_src, masked_sink, masked_weights, x_pos, y_pos, z_pos = create_masked_directed_energy_graph_from_mask(masked_array)

(256, 256, 256) (256, 256, 256)
256 256 256


/var/folders/0t/my3c3j251vl2s6y6tvgd80ph0000gn/T/ipykernel_19351/3695038853.py:5: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of -1 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  masked_array[b_arr_up == 0] = -1


Time taken to add vertices to coord_to_vertex: 76.75076198577881
Time taken to add edges to graph: 131.90148305892944
Time taken to add source and sink nodes: 1.5832099914550781


In [7]:
#plot masked array
print(masked_graph.num_vertices(), masked_graph.num_edges(), masked_src, masked_sink)
res_i = 0
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(masked_array[:,slice_index,:])
    elif axis == 2:
        plt.imshow(masked_array[:,:,slice_index])
    else:
        plt.imshow(masked_array[slice_index,:,:])
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

# Create a slider to browse through slices
interact(plot_slice, slice_index=IntSlider(min=0, max=masked_array.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

16777218 100140032 16777216 16777217


interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [8]:
#plot label ontop of raw data
res_i = 0
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(mark_boundaries_color(b_arr_up[:,slice_index,:], reduced_mask_data[res_i][:,slice_index,:]))
    elif axis == 2:
        plt.imshow(mark_boundaries_color(b_arr_up[:,:,slice_index], reduced_mask_data[res_i][:,:,slice_index]))
    else:
        plt.imshow(mark_boundaries_color(b_arr_up[slice_index,:,:], reduced_mask_data[res_i][slice_index,:,:]))
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

# Create a slider to browse through slices
interact(plot_slice, slice_index=IntSlider(min=0, max=b_arr_up.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [9]:
boundary_array, flow = calculate_seam_iter(masked_graph, masked_src, masked_sink, masked_weights, masked_array.shape[0], x_pos, y_pos, z_pos)

Time taken to calculate max flow boykov: 30.87933588027954
Time taken to calculate max flow: 0.6471571922302246 flow: 139426
Time taken to calculate max flow and min cut: 16.496016025543213
edges that cross the partition: 100140032
boundary vertices: 123460
Time taken to calculate seam: 1.5263051986694336
Time taken to convert boundary vertices to array: 0.4662909507751465


In [10]:
print("flow:", flow)
print("boundary_array:", np.sum(boundary_array))

flow: 139426
boundary_array: 65536


In [11]:
#plot full res boundary array
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(boundary_array[:,slice_index,:])
    elif axis == 2:
        plt.imshow(boundary_array[:,:,slice_index])
    else:
        plt.imshow(boundary_array[slice_index,:,:])
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

# Create a slider to browse through slices
interact(plot_slice, slice_index=IntSlider(min=0, max=boundary_array.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [12]:
#plot seam ontop of raw data
res_i = 0
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(mark_boundaries_color(boundary_array[:,slice_index,:], reduced_mask_data[res_i][:,slice_index,:]))
    elif axis == 2:
        plt.imshow(mark_boundaries_color(boundary_array[:,:,slice_index], reduced_mask_data[res_i][:,:,slice_index]))
    else:
        plt.imshow(mark_boundaries_color(boundary_array[slice_index,:,:], reduced_mask_data[res_i][slice_index,:,:]))
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

# Create a slider to browse through slices
interact(plot_slice, slice_index=IntSlider(min=0, max=b_arr_up.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [13]:
def multi_res_seam_iter(res_index, mask_array_data, b_arr_up):
    masked_array = mask_array_data[res_index].copy()
    masked_array[b_arr_up == 0] = -1
    stime= time.time()
    directed_graph, src, tgt, weights, x_pos, y_pos, z_pos = create_masked_directed_energy_graph_from_mask(masked_array)
    print(f"Time taken to create graph {res_index}:", time.time()-stime)
    boundary_array, flow = calculate_seam_iter(directed_graph, src, tgt, weights, masked_array.shape[0], x_pos, y_pos, z_pos)
    return boundary_array

def multi_res_seam_calculation(mask_array_data, res_index=3, upscale_factor=2, dilation_amount=1):
    stime= time.time()
    directed_graph, src, tgt, weights, x_pos, y_pos, z_pos = create_masked_directed_energy_graph_from_mask(mask_array_data[res_index])
    print(f"Time taken to create graph {res_index}:", time.time()-stime)
    boundary_array, flow = calculate_seam_iter(directed_graph, src, tgt, weights, mask_array_data[res_index].shape[0], x_pos, y_pos, z_pos)
    b_arr_up = upscale_and_dilate_3d(boundary_array, upscale_factor=2, dilation_amount=dilation_amount)
    for i in range(res_index-1, -1, -1):
        boundary_array = multi_res_seam_iter(i, mask_array_data, b_arr_up)
        b_arr_up = upscale_and_dilate_3d(boundary_array, upscale_factor=upscale_factor, dilation_amount=dilation_amount)
    return boundary_array



In [14]:
def remove_voxels(data_array, mask_array, boundary_array, num_voxels_to_remove=1, direction='x'):
    """
    Removes a specified number of voxels from each row in the given direction
    based on a boundary mask array.
    
    Parameters:
    data_array (np.ndarray): The original 3D array from which voxels are to be removed.
    boundary_array (np.ndarray): The boundary mask 3D array specifying the voxels to be removed.
    num_voxels_to_remove (int): Number of voxels to be removed from each row.
    direction (str): The direction in which voxels are to be removed ('x', 'y', 'z').
    
    Returns:
    np.ndarray: The resulting 3D array with the specified voxels removed.
    """
    assert data_array.shape == boundary_array.shape, f"Data array and boundary array must have the same shape. {data_array.shape} != {boundary_array.shape}"
    assert mask_array.shape == boundary_array.shape, f"Mask array and boundary array must have the same shape. {mask_array.shape} != {boundary_array.shape}"
    assert direction in ['x', 'y', 'z'], "Direction must be 'x', 'y', or 'z'."
    
    if direction == 'x':
        new_shape = (data_array.shape[0], data_array.shape[1], data_array.shape[2] - num_voxels_to_remove)
        result_data_array = np.zeros(new_shape, dtype=data_array.dtype)
        result_mask_array = np.zeros(new_shape, dtype=mask_array.dtype)
        for i in range(data_array.shape[0]):
            for j in range(data_array.shape[1]):
                data_row = data_array[i, j, :]
                boundary_row = boundary_array[i, j, :]
                keep_indices = np.where(boundary_row == 0)[0]
                keep_indices = keep_indices[:new_shape[2]]  # Only keep up to the new size
                result_data_array[i, j, :] = data_row[keep_indices][:new_shape[2]]
                result_mask_array[i, j, :] = mask_array[i, j, keep_indices][:new_shape[2]]
        
        # Create padded arrays with -1
        padded_data_array = -np.ones_like(data_array, dtype=data_array.dtype)
        padded_mask_array = -np.ones_like(mask_array, dtype=mask_array.dtype)
        
        # Copy the result arrays into the padded arrays
        padded_data_array[:, :, :new_shape[2]] = result_data_array
        padded_mask_array[:, :, :new_shape[2]] = result_mask_array
    
    elif direction == 'y':
        new_shape = (data_array.shape[0], data_array.shape[1] - num_voxels_to_remove, data_array.shape[2])
        result_data_array = np.zeros(new_shape, dtype=data_array.dtype)
        result_mask_array = np.zeros(new_shape, dtype=mask_array.dtype)
        for i in range(data_array.shape[0]):
            for k in range(data_array.shape[2]):
                data_row = data_array[i, :, k]
                boundary_row = boundary_array[i, :, k]
                keep_indices = np.where(boundary_row == 0)[0]
                keep_indices = keep_indices[:new_shape[1]]  # Only keep up to the new size
                result_data_array[i, :, k] = data_row[keep_indices][:new_shape[1]]
                result_mask_array[i, :, k] = mask_array[i, keep_indices, k][:new_shape[1]]
        
        # Create padded arrays with -1
        padded_data_array = -np.ones_like(data_array, dtype=data_array.dtype)
        padded_mask_array = -np.ones_like(mask_array, dtype=mask_array.dtype)
        
        # Copy the result arrays into the padded arrays
        padded_data_array[:, :new_shape[1], :] = result_data_array
        padded_mask_array[:, :new_shape[1], :] = result_mask_array
    
    elif direction == 'z':
        new_shape = (data_array.shape[0] - num_voxels_to_remove, data_array.shape[1], data_array.shape[2])
        result_data_array = np.zeros(new_shape, dtype=data_array.dtype)
        result_mask_array = np.zeros(new_shape, dtype=mask_array.dtype)
        for j in range(data_array.shape[1]):
            for k in range(data_array.shape[2]):
                data_row = data_array[:, j, k]
                boundary_row = boundary_array[:, j, k]
                keep_indices = np.where(boundary_row == 0)[0]
                keep_indices = keep_indices[:new_shape[0]]  # Only keep up to the new size
                result_data_array[:, j, k] = data_row[keep_indices][:new_shape[0]]
                result_mask_array[:, j, k] = mask_array[keep_indices, j, k][:new_shape[0]]
        
        # Create padded arrays with -1
        padded_data_array = -np.ones_like(data_array, dtype=data_array.dtype)
        padded_mask_array = -np.ones_like(mask_array, dtype=mask_array.dtype)
        
        # Copy the result arrays into the padded arrays
        padded_data_array[:new_shape[0], :, :] = result_data_array
        padded_mask_array[:new_shape[0], :, :] = result_mask_array
    
    return padded_data_array, padded_mask_array

In [22]:
res_index = 3
mask_array_data = reduced_mask_data
raw_array_data = reduced_raw_data[0]
num_seams_to_remove = 80

boundary_arrays = []

for i in range(num_seams_to_remove):
    boundary_array = multi_res_seam_calculation(mask_array_data, res_index, dilation_amount=1)
    mask_array_data, raw_array_data = remove_voxels(mask_array_data[0], raw_array_data, boundary_array)
    mask_array_data = coarsen_image(mask_array_data, 3)
    boundary_arrays.append(boundary_array)

    #TODO: remove seam from mask_array_data

32 32 32
Time taken to add vertices to coord_to_vertex: 0.13080501556396484
Time taken to add edges to graph: 0.23699140548706055
Time taken to add source and sink nodes: 0.01970505714416504
Time taken to create graph 3: 0.39046406745910645
Time taken to calculate max flow boykov: 0.04002189636230469
Time taken to calculate max flow: 0.011615276336669922 flow: 2275
Time taken to calculate max flow and min cut: 0.006891012191772461
edges that cross the partition: 188544
boundary vertices: 1615
Time taken to calculate seam: 0.0036809444427490234
Time taken to convert boundary vertices to array: 0.006754159927368164
64 64 64


/var/folders/0t/my3c3j251vl2s6y6tvgd80ph0000gn/T/ipykernel_19351/3957142680.py:3: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of -1 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  masked_array[b_arr_up == 0] = -1


Time taken to add vertices to coord_to_vertex: 1.098560094833374
Time taken to add edges to graph: 1.9038548469543457
Time taken to add source and sink nodes: 0.08116507530212402
Time taken to create graph 2: 3.100813865661621
Time taken to calculate max flow boykov: 0.2896409034729004
Time taken to calculate max flow: 0.04303121566772461 flow: 6535
Time taken to calculate max flow and min cut: 0.0706169605255127
edges that cross the partition: 1540352
boundary vertices: 6422
Time taken to calculate seam: 0.023509979248046875
Time taken to convert boundary vertices to array: 0.025722980499267578
128 128 128
Time taken to add vertices to coord_to_vertex: 8.687899112701416
Time taken to add edges to graph: 15.729487895965576
Time taken to add source and sink nodes: 0.3307068347930908
Time taken to create graph 1: 24.870909214019775
Time taken to calculate max flow boykov: 2.7160048484802246
Time taken to calculate max flow: 0.15847992897033691 flow: 24322
Time taken to calculate max flow

In [23]:
print(mask_array_data[0].shape, raw_array_data.shape)

(256, 256, 256) (256, 256, 256)


In [24]:
#save results
# Create output directory if it doesn't exist
output_dir = os.path.join(os.getcwd(), 'output/densified_cubes')
os.makedirs(output_dir, exist_ok=True)

# Get the current timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save mask_array_data[0] as NRRD with a timestamp
mask_nrrd_path = os.path.join(output_dir, f'densified_label_{timestamp}.nrrd')
nrrd.write(mask_nrrd_path, mask_array_data[0])
print(f"Saved mask_array_data[0] to {mask_nrrd_path}")

# Save raw_array_data as NRRD with a timestamp
raw_nrrd_path = os.path.join(output_dir, f'densified_data_{timestamp}.nrrd')
nrrd.write(raw_nrrd_path, raw_array_data)
print(f"Saved raw_array_data to {raw_nrrd_path}")

Saved mask_array_data[0] to /Users/jamesdarby/Documents/VesuviusScroll/GP/3D_sheet_carving/output/densified_cubes/densified_label_20240607_060716.nrrd
Saved raw_array_data to /Users/jamesdarby/Documents/VesuviusScroll/GP/3D_sheet_carving/output/densified_cubes/densified_data_20240607_060716.nrrd


In [25]:
#plot seam ontop of mask array
res_i = 0
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(mark_boundaries_color(raw_array_data[:,slice_index,:], mask_array_data[res_i][:,slice_index,:]))
    elif axis == 2:
        plt.imshow(mark_boundaries_color(raw_array_data[:,:,slice_index], mask_array_data[res_i][:,:,slice_index]))
    else:
        plt.imshow(mark_boundaries_color(raw_array_data[slice_index,:,:], mask_array_data[res_i][slice_index,:,:]))
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

# Create a slider to browse through slices
interact(plot_slice, slice_index=IntSlider(min=0, max=boundary_array.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [26]:
#plot boundary array, aka seam to remove
res_i = 0
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(boundary_array[:,slice_index,:])
    elif axis == 2:
        plt.imshow(boundary_array[:,:,slice_index])
    else:
        plt.imshow(boundary_array[slice_index,:,:])
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

# Create a slider to browse through slices
interact(plot_slice, slice_index=IntSlider(min=0, max=boundary_array.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [ ]:
#plot seam ontop of mask array
res_i = 0
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(mark_boundaries_color(boundary_array[:,slice_index,:], reduced_mask_data[res_i][:,slice_index,:]))
    elif axis == 2:
        plt.imshow(mark_boundaries_color(boundary_array[:,:,slice_index], reduced_mask_data[res_i][:,:,slice_index]))
    else:
        plt.imshow(mark_boundaries_color(boundary_array[slice_index,:,:], reduced_mask_data[res_i][slice_index,:,:]))
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

# Create a slider to browse through slices
interact(plot_slice, slice_index=IntSlider(min=0, max=boundary_array.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))